In [1]:
# General Dependencies

import os
import math
import numpy as np
import torch

# Lines below are reloading modules every time you run a cell,
# such that this cell doesn't have to be re-run every time something
# changes in the source code.
#%load_ext autoreload 
#%autoreload 2

In [3]:
"""
Run from Colab or local
"""

try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    ROOT_PATH = "/content/gdrive/MyDrive/AAAFormer"
    DATA_PATH = os.path.join(ROOT_PATH, "Datasets")
    
    %cd ./gdrive/MyDrive/AAAFormer

except:
    ROOT_PATH = os.curdir
    DATA_PATH = "../Datasets"




In [4]:
# Source dependencies
from data.dataset import FSSDataset
from common.vis import Visualizer
from common.evaluation import Evaluator

## Prepare PASCAL-$5^i$ Dataset

In [13]:
# STEP 1: Download PASCAL VOC2012 devkit (train/val data): (uncomment lines below)
# ------------------------------------------------------------------------------
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
#!tar -xvf 'VOCtrainval_11-May-2012.tar' -C ./Datasets/ 

# (or instead of wget, use directly the link to download)
# STEP 2: Place "VOC2012" folder from downloaded "VOCdevkit" folder under a "Datasets" folder.
# ------------------------------------------------------------------------------
#!mv Datasets/VOCdevkit/VOC2012 Datasets/VOC2012

# STEP 3: Download extended annotations from here
#!gdown https://drive.google.com/file/d/10zxG2VExoEZUeyQl_uXga2OWHjGeZaf2/view

# STEP 4: Extract Put the downloaded extension under "Datasets/VOC2012/"


'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
# Dataset initialization 
# TODO: does paper mention about image size? TODO: yes, 473
# TODO: they also do data aug...
# TODO: add disclaimer to dataset files https://github.com/juhongm999/hsnet/blob/main/train.py
FSSDataset.initialize(img_size=128, datapath=DATA_PATH, use_original_imgsize=False)
dataloader_trn = FSSDataset.build_dataloader(benchmark='pascal', bsz=4, nworker=1, fold=0, split='trn')
dataloader_val = FSSDataset.build_dataloader(benchmark='pascal', bsz=4, nworker=1, fold=0, split='val')

Total (trn) images are : 11394
Total (val) images are : 346


## deneme

In [6]:
Visualizer.initialize(True)
Evaluator.initialize()

In [7]:
""" Code below visualizes prediction (to be used in testing)

"""
for idx, batch in enumerate(dataloader_trn):
        # 1. Hypercorrelation Squeeze Networks forward pass
        #batch = utils.to_cuda(batch)
        #pred_mask = model.module.predict_mask_nshot(batch, nshot=nshot)

        #assert pred_mask.size() == batch['query_mask'].size()
        pred_mask = batch['query_mask']

        # 2. Evaluate prediction
        area_inter, area_union = Evaluator.classify_prediction(pred_mask.clone(), batch)
        
        #average_meter.update(area_inter, area_union, batch['class_id'], loss=None)
        #average_meter.write_process(idx, len(dataloader), epoch=-1, write_batch_idx=1)

        # Visualize predictions
        if Visualizer.visualize:
            Visualizer.visualize_prediction_batch(batch['support_imgs'], batch['support_masks'],
                                                  batch['query_img'], batch['query_mask'],
                                                  pred_mask, batch['class_id'], idx,
                                                  area_inter[1].float() / area_union[1].float())

        break   # TODO: delete this break to run visualization for full dataset

## Extract features

In [9]:
from model import feature

In [10]:
from torchvision.models import resnet
from functools import reduce
from operator import add

for idx, batch in enumerate(dataloader_trn):

    
    query_img = batch['query_img']
    
    backbone = resnet.resnet50(pretrained=True)
    
    
    nbottlenecks = [3, 4, 6, 3] # Resnet50
    
    feat_ids = list(range(4, 17))
    bottleneck_ids = reduce(add, list(map(lambda x: list(range(x)), nbottlenecks)))
    
    lids = reduce(add, [[i + 1] * x for i, x in enumerate(nbottlenecks)])
    stack_ids = torch.tensor(lids).bincount().__reversed__().cumsum(dim=0)[:3]
    backbone.eval()
    
    query_feats = feature.extract_feat_res(query_img, backbone,  feat_ids, bottleneck_ids, lids)
    
    break

In [11]:
print(len(query_feats))
 


13


In [12]:
for feat in query_feats:
    print(feat.shape)

torch.Size([4, 512, 16, 16])
torch.Size([4, 512, 16, 16])
torch.Size([4, 512, 16, 16])
torch.Size([4, 512, 16, 16])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 2048, 4, 4])
torch.Size([4, 2048, 4, 4])
torch.Size([4, 2048, 4, 4])


## Initial Agent Tokens

### TODO: if this implementation is OK, move it to a source file.

In [131]:
# num_tokens is K in the algorithm 1 (not the K of K-shot images)
# Please refer to Algorithm 1 given in Supplementary Material
# to match the notation of variables in the comments.
# We assume X, locations of foreground pixels, is the locations of
# f_s, i..e foreground support pixel features.


# Shapes:
# X --> [batchsize, num_foreground_pixels, 2]
# L --> [batchsize, num_background_pixels, 2] last dimension is (x,y) location
# f_s --> [batchsize, h, w, c] and h*w = num_foreground_pixels
def init_agent_tokens(num_tokens, X, L, f_s):
  
    # Compute euclidean distance between every pair
    # (foreground_pixel, bacground_pixel)
    # in total, |X| x |L| pairs
    dists_batch = torch.cdist(X, L)   # Get all the distances for K support ims

    tokens = torch.empty((X.shape[0], num_tokens, f_s.shape[-1]))
    L_new = []
    # TODO: can we compute this jointly for all images in a batch?
    for i in range(X.shape[0]):
        L_single = L[i,:,:]      # L for a single image in a batch

        for k in range(num_tokens):
            dists = dists_batch[i]

            # See line 3 of Algorithm 1 in Supplementary Material:
            # for a specific location x, min distance between x and all other locations in L
            d_x, d_x_ind = torch.min(dists, dim=1)  

            # We don't care about the actual distance value, so it is named as _
            # we care about which location has the furthest distance p* 
            _ , p_ind = torch.max(d_x, dim=0)

            p_furthest = X[i, p_ind, :]      # This is a location (x,y) of a pixel
            p_star = p_furthest.unsqueeze(0) # [2] --> [1,2] 
            L_single = torch.cat([L_single, p_star], dim=0) # L = (B) U (P), see line 5 in Algorithm 1

            f_a_k = f_s[i, p_furthest.data[0].long().item(), p_furthest.data[1].long().item(), :]
            
            tokens[i,k,:] = f_a_k
            
        L_new.append(L_single)
    
    return tokens
    
    

In [132]:
# TESTING initial tokens with random inputs ...
X = torch.randint(low=0, high=1,size=(2, 4, 2)).float()
L = torch.randint(low=0, high=2,size=(2, 6, 2)).float()
num_tokens = K = 5
f_s = torch.randn(2, 2, 2, 3)

tokens = init_agent_tokens(K, X, L, f_s) # every token has [K,c] dim for every sample in a batch
print(tokens.shape)

torch.Size([2, 5, 3])


In [76]:

for idx, batch in enumerate(dataloader_trn):
    
    query_img = batch['query_img']
    
    # get foreground pixels
    # get background pixels
    # get agent tokens
    
    break